# Redes de Camada Única

$$f: \mathbb{R}^{m \times n} \rightarrow \mathbb{R}^{m \times o}$$

Em uma rede de camada única, temos duas matrizes $x$ e $y$ de valores. A matriz $x$ possui as dimensões de $m$ por $n$, onde $m$ diz sobre a quantidade de instâncias (linhas) e $n$ a quantidade de entradas. A matriz $y$, por sua vez, possui a mesma quantidade de instâncias de $x$, porém a quantidade de saídas pode ser diferente, sendo definida como $o$.

Dessa forma, para cada $x^{m \times n}$, existe um $y^{m \times o}$:

$$\begin{bmatrix}
x_{11} & x_{12} & x_{13} & x_{14}\\
x_{21} & x_{22} & x_{23} & x_{24}\\
x_{31} & x_{32} & x_{33} & x_{44}
\end{bmatrix}
\rightarrow
\begin{bmatrix}
y_{11} & y_{12}\\
y_{21} & y_{22}\\
y_{31} & y_{32}
\end{bmatrix}
$$

Neste caso

$$m = 3 ; n = 4; o = 2$$

In [ ]:
%pip install numpy pandas matplotlib scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Problema a ser resolvido

<div align="center">
<img src="https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Machine+Learning+R/iris-machinelearning.png" height="200" width="450" />

<img src="https://ars.els-cdn.com/content/image/3-s2.0-B9780128147610000034-f03-01-9780128147610.jpg?_" height="250" />
</div>

A flor Íris possui três tipos:
- Íris setosa
- Íris versicolor
- Íris virgínica

Para classificar a flor, são utilizadas variáveis como a largura e tamanho da pétala e sépala. Dessa forma, o conjunto de valores $x$ se dá por:

$$x = [sl, sw, pl, pw]$$

Onde:

- $sl \in \mathbb{R^{*}_{+}}$: Tamanho da sépala
- $sw \in \mathbb{R^{*}_{+}}$: Largura da sépala
- $pl \in \mathbb{R^{*}_{+}}$: Largura da pétala
- $pw \in \mathbb{R^{*}_{+}}$: Tamanho da pétala

A partir desse valor de $x$, é gerado três tipos de valores $y$ para identificar o tipo da íris:

$$y = [ise, ive, ivi]$$

Onde:

- $ise \in \{0, 1\}$: Íris setosa
- $ive \in \{0, 1\}$: Íris versicolor
- $ivi \in \{0, 1\}$: Íris virgínica

In [ ]:
dataset = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', 
                      names=["sepal_length","sepal_width", "petal_length", "petal_width", "class"])

dataset['Iris_Setosa'] = dataset['class'] == 'Iris-setosa'
dataset['Iris_Versicolor'] = dataset['class'] == 'Iris-versicolor'
dataset['Iris_Virginica'] = dataset['class'] == 'Iris-virginica'

X = dataset[['sepal_length','sepal_width','petal_length','petal_width']].values
Y = dataset[['Iris_Setosa','Iris_Versicolor','Iris_Virginica']].values
Y = np.where(Y == True, 1, 0)

## Amostra de dados

Uma boa prática para treinar redes neurais é separar uma amostra da população total de dados apenas para o treinamento e uma outra amostra apenas para os testes da rede.

Dessa forma, separamos 30% dos dados carregados acima para teste, enquanto os outros 70% do dados serão utilizados para o treinamento da rede.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

train = { "x": X_train, "y": Y_train, "n": len(X_train) }
test  = { "x": X_test, "y": Y_test, "n": len(X_test) }

## Parâmetros

### Hiper-parâmetros

O conjunto de hiper-parâmetros é dado por valores definidos manualmente por nós humanos.

#### Taxa de aprendizado

A taxa de aprendizado é uma constante real que define o tamanho dos ajustes feitos pela função de aprendizado. Valores maiores representam mudanças mais bruscas na atualização dos pesos .

$$\alpha = x \in \mathbb{R}$$



In [ ]:
a = 0.02

#### Quantidade de entradas, saídas e instâncias

**Instâncias ($m$)**: A quantidade de instâncias diz respeito a quantidade de linhas totais que a matriz $x$ (ou $y$) tem.

**Entradas ($n$)**: A quantidade de entradas é a quantidade de valores em cada linha de $x$, nesse caso são 4.

**Saídas ($o$)**: A quantidade de saídas é a quantidade de valores em cada linha de $y$, nesse caso são 3.


In [ ]:
m, n = X.shape
_, o = Y.shape

### Parâmetros

Os parâmetros consistem nos valores que a rede neural vai calcular durante o seu treinamento, no caso os pesos e os termos de viés.

#### Pesos ($w$)

$$w \in \mathbb{R}^{o \times n}$$

$$\begin{bmatrix}
w_{11} & w_{12} & ... & w_{1n}\\
w_{21} & w_{22} & ... & x_{2n}\\
w_{o1} & w_{o2} & ... & w_{on}
\end{bmatrix}$$

Os pesos são valores inicializados de forma aleatória e que serão atualizados por meio da função de aprendizado.

In [ ]:

w = [np.random.normal(0, 0.5, n) for _ in range(o)]

#### Viés ($b$)

$$
b \in \mathbb{R}^o
$$
$$
b = \begin{bmatrix}
b_1 \\
b_2 \\
\vdots \\
b_o
\end{bmatrix}$$

Assim como nos pesos, os termos de bias serão atualizados conforme erros acontecidos durante a função de inferência na função de aprendizado. Porém, como o $b$ não está diretamente relacionado ao $x$, logo sua matriz tem apenas uma coluna e $o$ linhas.

In [ ]:
b = [np.random.normal(0, 0.5, 1) for _ in range(o)]

## Função de ativação $(\sigma)$

$$\sigma(y, \hat{y}) = \frac{1}{1 + e^{-x}}$$

A função de ativação tem como objetivo classificar valores que foram resultados da função de inferência. Neste caso, utilizamos a função Sigmoide $\sigma$ para realizar essa classificação.

In [ ]:
def sigmoide(x) -> float:
  return 1/(1 + np.exp(-x))

## Função de custo ($L$)

A função de custo visa calcular o erro $e$ entre $y$ e $\hat{y}$.

### Mean Square Error

$$L(y, \hat{y}) = \frac{(y - \hat{y})^2}{2}$$

Neste caso, estamos utilizando a função MSE que calcula o erro médio quadrático entre os valores de $y$ e $\hat{y}$.

In [ ]:
def mse(y: float, y_hat: float) -> float:
  return ((y - y_hat)**2)/2

## Função de inferência ($f(x)$)

$$\hat{y_{j}} = \sum^{n}_{i=1} w_{ij} \cdot x_{i} + b_{j}$$

A função de inferência tem como objetivo replicar o comportamento da função geradora $G(x)$. Porém, como essa rede neuronal possui uma camada, o somatório se dá seguinte forma:

- $j$: Índice do neurônio de saída (exemplo: $(y_1, y_2, \ldots, y_o )$).
- $i$: Índice do neurônio de entrada (exemplo: $( x_1, x_2, \ldots, x_n )$).
- $w_{ji}$: Peso que conecta o $i$-ésimo neurônio de entrada ao $j$-ésimo neurônio de saída.
- $b_j$: Viés específico do $j$-ésimo neurônio de saída.


In [ ]:
def inference(x: list, w: list, b: float) -> float:
  return np.dot(x, w) + b

## Função de aprendizado ($F(x)$)

$$F(x) =
      \begin{cases}
      w_i^{t+1} = w_i^t - \alpha \cdot \frac{\partial \varepsilon}{\partial w_i} \\
      b^{t+1} = b^t - \alpha \cdot \frac{\partial \varepsilon}{\partial b}
      \end{cases}$$

A função de aprendizagem em redes neurais é baseada no ajuste dos parâmetros do modelo (como pesos e viés) para reduzir o erro entre a previsão do modelo e o valor real. O objetivo é minimizar a função de custo, que, neste caso, é a *Mean Squared Error* (MSE), através do uso do gradiente descendente.

### Gradiente descendente $\nabla$

A técnica do gradiente descendente visa descobrir o impacto do erro $\varepsilon$ em relação ao $\hat{y}$. Porém, como essas variáveis não estão relacionas diretamente, temos que utilizar a derivada parcial.

$$L(x) \text{ ou } \varepsilon \leftarrow \hat{y} \text{ ou } f(x)  \leftarrow w,b $$

Dessa forma, para calcular o erro ($L(x) \text{ ou } \varepsilon$) dependemos da estimativa ($\hat{y}$) e para calcular a estimativa dependemos dos pesos e termo de viés ($w$ e $b$). 

Ou seja, a derivada parcial se dá por:

$$
\frac{\partial\varepsilon}{\partial\theta} = \frac{\partial{\varepsilon}}{\partial\sigma} \cdot \frac{\partial\sigma}{\partial\hat{y}} \cdot \frac{\partial\hat{y}}{\partial\theta}
$$
$$
\frac{\partial\varepsilon}{\partial\sigma} = \left(\frac{(y - \hat{y})^2}{2}\right)' = \left(0.5 \cdot (y - \hat{y})^2\right)' = y - \hat{y}
$$
$$
\frac{\partial\varepsilon}{\partial\sigma} = \left(\frac{1}{1 + e^{-x}}\right)' = \frac{1}{1 + e^{-x}} \cdot (1 - y)
$$
$$
\frac{\partial\hat{y}}{\partial\theta} = \left(w \cdot x + b\right)' = x \cdot 1 = x
$$

Porém, a derivada dessas funções aponta para onde a função está crescendo, mas como queremos otimizar a atualização dos pesos, devemos multiplicar pelo inverso da derivada parcial:

$$-\left(\frac{\partial\varepsilon}{\partial\theta}\right)$$

In [ ]:
def sigmoide_derivate(x) -> float:
  y = sigmoide(x)
  return y * (1 - y)

def mse_derivate(y: float, y_hat: float) -> float:
  return y - y_hat

def learn(x: list, y: float, y_hat: float, w: float, b: float, a: float) -> float:
  error = mse(y, y_hat)
  y_hat_activation = sigmoide(y_hat)
  delta = mse_derivate(y, y_hat_activation) * sigmoide_derivate(y_hat)

  w += a * -delta * -x
  b += a * -delta

  return w, b, error

In [ ]:
x, y, n = train['x'], train['y'], train['n']

epochs = 100
training_errors = []

for epoch in range(epochs):
  for i in range(n):
    random = np.random.randint(n)
    xi, yi = x[random], y[random]
    instance_errors = []

    for output in range(o):
      y_hat = inference(xi, w[output], b[output])
      w[output], b[output], error = learn(xi, yi[output], y_hat, w[output], b[output], a)
      instance_errors.append(error)

    training_errors.append(np.mean(instance_errors))

In [ ]:
x, y, n = test['x'], test['y'], test['n']

y_hat = []

for instance in range(n):
  tmp = [sigmoide(inference(x[instance], w[output], b[output])) for output in range(o)]
  tmp = np.where(tmp == max(tmp), 1, 0)
  y_hat.append(tmp)

y_hat = np.array(y_hat)

accuracy = accuracy_score(y.argmax(axis=1), y_hat.argmax(axis=1))

print(f'Accuracy: {accuracy*100:.2f}%')